In [ ]:
import matplotlib.pyplot as plt
from skimage import io
from skimage import color
from skimage.transform import resize
import math
from skimage.feature import hog
import numpy as np
import pandas as pd
from PIL import Image
from skimage import data, exposure
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler

In [ ]:
# self-written scripts
import sys
sys.path.insert(0, 'Python_Scripts')

import util

In [ ]:
!pip install seaborn

In [ ]:
!pip install imblearn

In [ ]:
!pip install scikit-learn

In [ ]:
df = pd.read_csv('data/train_complete.csv')

In [ ]:
hog = pd.read_csv('data/train_HOG.csv')

In [ ]:
# Count occurrences of `ImageId` in df
df['count'] = df.ImageId.apply(lambda x: df['ImageId'].value_counts()[x])

In [ ]:
# isolate `ImageIds` for images with defect
single_defects = df.query('count > 1').ImageId.to_numpy()

indices = []
for idx, row in hog.iterrows():
    if row.ImageId in single_defects:
        indices.append(idx)
        
hog.drop(indices, inplace=True)

In [ ]:
hog_complete = hog.merge(df[['ClassId','ImageId']], on = 'ImageId')
hog_complete.head()

In [ ]:
#Split Hog_complete in X und Y
X = hog_complete.drop(['ClassId','ImageId'], axis =1)
y = hog_complete['ClassId']
y

In [ ]:
#Oversampling
ros = RandomOverSampler(random_state=42)
X_ros, y_ros = ros.fit_resample(X, y)

In [ ]:
from sklearn.model_selection import train_test_split
# Split without oversampled data
#X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,stratify=y)
# Split with oversampling
X_train, X_test, y_train, y_test = train_test_split(X_ros,y_ros,test_size=0.2,stratify=y_ros)
print('Training data and target sizes: \n{}, {}'.format(X_train.shape,y_train.shape))
print('Test data and target sizes: \n{}, {}'.format(X_test.shape,y_test.shape))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics


In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
test_accuracy = []
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

classifier = KNeighborsClassifier(n_neighbors=3,algorithm='brute')
classifier.fit(X_train_scaled, y_train)

y_pred = classifier.predict(X_test_scaled)
test_accuracy = classifier.score((X_test_scaled), y_test)
print(test_accuracy)

In [ ]:
print(metrics.confusion_matrix(y_test,y_pred))
# Print confusion matrix
sns.heatmap(metrics.confusion_matrix(y_test, y_pred), annot=True, cmap='YlGn');

In [ ]:
print(metrics.classification_report(y_test, y_pred))

# results without oversampling   
    
    precision    recall  f1-score   support

           0       0.75      0.89      0.81      1181
           1       0.48      0.64      0.55       154
           2       0.32      0.72      0.44        39
           3       0.87      0.67      0.76       952
           4       0.33      0.01      0.02       103

    accuracy                           0.75      2429
   macro avg       0.55      0.58      0.52      2429
weighted avg       0.75      0.75      0.73      2429

---

## SURF

In [ ]:
surf = pd.read_csv('data/train_surf.csv')

In [ ]:
util.isolate_single_defects(surf)